In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

###***-: UPLODING '.csv' FILE FOR USE :-***

In [ ]:
from google.colab import files
print("Please upload your CSV file:")
uploaded = files.upload()
sales_data = next(iter(uploaded))

### ***-: CHENGING '.csv' INTO DATAFRAME :-***

In [ ]:
df = pd.read_csv("customer_data.csv")

df.head()

### ***-: DATA UNDERSTANDING & INITIAL CHECKS  :-***

In [ ]:
#1. Basic Shape & Preview :-

print("Rows:", df.shape[0])
print("Columns:", df.shape[1])

df.head()

In [ ]:
#2. Column-Level Overview :-

df.info()

In [ ]:
#3. Check Missing Values :-

df.isna().sum()

In [ ]:
#4. Numerical Summary :-

df.describe()

In [ ]:
#5/1. Categorical Overview :-

df["funnel_stage"].value_counts()

In [ ]:
#5/2. Categorical Overview :-

df["channel"].value_counts()

In [ ]:
#5/3. Categorical Overview :-

df["country"].value_counts()

In [ ]:
#6/1. Customer-Level Reality Check :-

df["customer_id"].nunique()

In [ ]:
#6/2. Customer-Level Reality Check :-

df.groupby("customer_id").size().describe()

### ***-: DATA QUALITY ISSUES & BUSINESS ASSUMPTIONS :-***

*Data Quality Assumptions*
- Missing signup dates are replaced using the customer’s first observed order date.
- Negative revenue values are treated as refunds and excluded from CLV calculations.
- Only purchase-related funnel stages are used for revenue and CLV analysis.
- Customer lifetime is calculated as the duration between first and last purchase dates.
- CLV is computed using total positive revenue per customer.


### ***-: DATA CLEANING & FEATURE PREPARATION :-***

In [ ]:
#1. Preserve Raw Data :-

df_raw = df.copy()

In [ ]:
#2. Fix Missing Signup Dates :-


df['signup_date'] = pd.to_datetime(df['signup_date'], errors='coerce')
df['order_date'] = pd.to_datetime(df['order_date'], errors='coerce')

df['signup_date'] = df.groupby('customer_id')['signup_date'] \
    .transform(lambda x: x.fillna(x.min()))

df['signup_date'].isna().sum()

In [ ]:
#3. Separate Purchase Events :-

purchase_df = df[df["funnel_stage"].isin(["Purchase", "Repeat Purchase"])].copy()

purchase_df["funnel_stage"].value_counts()

In [ ]:
#4. Remove Negative Revenue for CLV :-

purchase_df = purchase_df[purchase_df["revenue"] > 0]

purchase_df["revenue"].describe()

In [ ]:
#5. Create Customer-Level CLV Features :-

clv_df = purchase_df.groupby("customer_id").agg(
    total_revenue=("revenue", "sum"),
    total_orders=("order_id", "count"),
    first_purchase=("order_date", "min"),
    last_purchase=("order_date", "max")
).reset_index()

In [ ]:
#6. Calculate Customer Lifetime (Days) :-

clv_df["customer_lifetime_days"] = (
    clv_df["last_purchase"] - clv_df["first_purchase"]
).dt.days

In [ ]:
#7. Average Order Value (AOV) :-

clv_df["avg_order_value"] = (
    clv_df["total_revenue"] / clv_df["total_orders"]
)

In [ ]:
#8. Final CLV Metric :-

clv_df["CLV"] = clv_df["total_revenue"]

In [ ]:
#9. Add Signup Cohort Feature :-

clv_df = clv_df.merge(
    df[["customer_id", "signup_date"]].drop_duplicates(),
    on="customer_id",
    how="left"
)

clv_df["signup_month"] = clv_df["signup_date"].dt.to_period("M")

In [ ]:
#10. Final Dataset Check :-

clv_df.info()

In [ ]:
# // Optional Step //

clv_df.head()

### ***-: CUSTOMER LIFETIME VALUE [CLV] ANALYSIS :-***

In [ ]:
#1. Overall CLV Distribution :-

clv_df["CLV"].describe()

In [ ]:
#2. CLV Distribution Visualization :-

plt.figure(figsize=(10,5))
sns.histplot(clv_df["CLV"], bins=50, kde=True)
plt.title("Customer Lifetime Value Distribution")
plt.xlabel("CLV")
plt.ylabel("Number of Customers")
plt.show()

In [ ]:
#3. Identify High-Value Customers :-

clv_df["clv_percentile"] = clv_df["CLV"].rank(pct=True)

clv_df["clv_segment"] = pd.cut(
    clv_df["clv_percentile"],
    bins=[0, 0.5, 0.8, 0.95, 1.0],
    labels=["Low Value", "Mid Value", "High Value", "VIP"]
)

clv_df["clv_segment"].value_counts()

In [ ]:
#4. Revenue Contribution by CLV Segment :-

clv_df.groupby("clv_segment")["CLV"].sum().sort_values(ascending=False)

In [ ]:
#5. CLV vs Customer Behavior :-

# CLV vs Number of Orders
plt.figure(figsize=(8,5))
sns.scatterplot(
    data=clv_df,
    x="total_orders",
    y="CLV",
    hue="clv_segment",
    alpha=0.7
)
plt.title("CLV vs Total Orders")
plt.show()


# CLV vs Customer Lifetime
plt.figure(figsize=(8,5))
sns.scatterplot(
    data=clv_df,
    x="customer_lifetime_days",
    y="CLV",
    hue="clv_segment",
    alpha=0.7
)
plt.title("CLV vs Customer Lifetime")
plt.show()

In [ ]:
#6. CLV by Acquisition Channel :-

channel_clv = purchase_df.merge(
    clv_df[["customer_id", "CLV"]],
    on="customer_id",
    how="left"
)

channel_clv.groupby("channel")["CLV"].mean().sort_values(ascending=False)

### ***-: CONCLUSION :-***

*CLV Analysis Insights*

- Customer Lifetime Value distribution is highly skewed, with a small proportion of customers contributing the majority of total revenue.
- VIP and High-Value customers generate a disproportionately large share of lifetime revenue.
- CLV is more strongly influenced by purchase frequency than by customer lifespan alone.
- Acquisition channels differ significantly in the quality of customers they attract, not just volume.
- Retaining high-CLV customers should be a top priority for revenue stability.


### ***-: COHORT ANALYSIS :-***

In [ ]:
#1. Prepare Cohort Data :-

cohort_df = purchase_df.copy()

cohort_df["order_month"] = cohort_df["order_date"].dt.to_period("M")

cohort_df = cohort_df.merge(
    clv_df[["customer_id", "signup_month"]],
    on="customer_id",
    how="left"
)

cohort_df.head()

In [ ]:
#2. Calculate Cohort Index :-

def cohort_index(row):
    return (row["order_month"].year - row["signup_month"].year) * 12 + \
           (row["order_month"].month - row["signup_month"].month)

cohort_df["cohort_index"] = cohort_df.apply(cohort_index, axis=1)

In [ ]:
#3. Build Cohort Table :-

cohort_counts = cohort_df.groupby(
    ["signup_month", "cohort_index"]
)["customer_id"].nunique().reset_index()

cohort_counts.head()

In [ ]:
#4. Create Cohort Retention Matrix :-

cohort_pivot = cohort_counts.pivot(
    index="signup_month",
    columns="cohort_index",
    values="customer_id"
)

cohort_sizes = cohort_pivot.iloc[:, 0]
retention_matrix = cohort_pivot.divide(cohort_sizes, axis=0)

In [ ]:
#5. Visualize Retention Heatmap :-

plt.figure(figsize=(14,8))
sns.heatmap(
    retention_matrix,
    annot=True,
    fmt=".0%",
    cmap="Blues"
)
plt.title("Customer Retention Cohort Analysis")
plt.xlabel("Months Since Signup")
plt.ylabel("Signup Cohort")
plt.show()

In [ ]:
#6. Retention Summary Metrics :-

retention_summary = retention_matrix.mean()
retention_summary

### ***-: CONCLUSION :-***

*Cohort Retention Insights*

- Customer retention drops significantly after the first month, indicating early churn risk.
- Later-month retention stabilizes, suggesting a core group of loyal customers.
- Some signup cohorts consistently outperform others, indicating differences in acquisition quality.
- Improving early-stage engagement could significantly increase long-term customer value.

### ***-: FUNNEL ANALYSIS :-***

In [ ]:
#1. Prepare Funnel Data :-

funnel_df = df.copy()
funnel_df["funnel_stage"].value_counts()

In [ ]:
#2. Define Funnel Order :-

funnel_order = ["Visit", "Signup", "Purchase", "Repeat Purchase"]

In [ ]:
#3. Customer-Level Funnel Progression :-

funnel_counts = (
    funnel_df.groupby("funnel_stage")["customer_id"]
    .nunique()
    .reindex(funnel_order)
)

funnel_counts

In [ ]:
#4. Conversion Rates Between Stages :-

funnel_conversion = funnel_counts / funnel_counts.shift(1)
funnel_conversion.iloc[0] = 1  # First stage baseline

funnel_table = pd.DataFrame({
    "Customers": funnel_counts,
    "Conversion_Rate": funnel_conversion
})

funnel_table

In [ ]:
#5. Funnel Visualization :-

plt.figure(figsize=(8,5))
sns.barplot(
    x=funnel_table.index,
    y=funnel_table["Customers"]
)
plt.title("Customer Funnel Drop-offs")
plt.ylabel("Number of Customers")
plt.show()

In [ ]:
#6. Funnel Analysis by Acquisition Channel :-

channel_funnel = (
    funnel_df.groupby(["channel", "funnel_stage"])["customer_id"]
    .nunique()
    .reset_index()
)


channel_pivot = channel_funnel.pivot(
    index="channel",
    columns="funnel_stage",
    values="customer_id"
).reindex(columns=funnel_order)

channel_pivot

In [ ]:
#7. Channel-wise Conversion Rates :-

channel_conversion = channel_pivot.div(channel_pivot.shift(axis=1), axis=0)
channel_conversion

### ***-: CONCLUSION :-***

*Funnel Analysis Insights*

- The largest customer drop-off occurs between Visit and Signup stages.
- Conversion from Purchase to Repeat Purchase is relatively low, indicating retention opportunities.
- Acquisition channels vary significantly in funnel efficiency, not just customer volume.
- Improving early-stage onboarding and post-purchase engagement can substantially improve overall funnel performance.

### ***-: ADVANCED INSIGHT [CLV × RETENTION × FUNNEL] :-***

In [ ]:
#1. CLV by Signup Cohort :-

cohort_clv = clv_df.groupby("signup_month")["CLV"].mean()

cohort_clv

In [ ]:
#2. Visualize :-

plt.figure(figsize=(12,5))
cohort_clv.plot(marker="o")
plt.title("Average CLV by Signup Cohort")
plt.xlabel("Signup Month")
plt.ylabel("Average CLV")
plt.show()

In [ ]:
#3. CLV vs Retention Strength :-

month_1_retention = retention_matrix[1]

cohort_analysis = pd.DataFrame({
    "avg_clv": cohort_clv,
    "month_1_retention": month_1_retention
}).dropna()

cohort_analysis

In [ ]:
#4. Visualize :-

plt.figure(figsize=(8,5))
sns.scatterplot(
    data=cohort_analysis,
    x="month_1_retention",
    y="avg_clv"
)
plt.title("CLV vs Month-1 Retention by Cohort")
plt.show()

In [ ]:
#5. Funnel Stage Impact on CLV :-

funnel_clv = purchase_df.merge(
    clv_df[["customer_id", "CLV"]],
    on="customer_id",
    how="left"
)

funnel_clv.groupby("funnel_stage")["CLV"].mean()

In [ ]:
#6. Channel Quality: Volume vs Value :-

channel_value = purchase_df.merge(
    clv_df[["customer_id", "CLV"]],
    on="customer_id",
    how="left"
)

channel_summary = channel_value.groupby("channel").agg(
    avg_clv=("CLV", "mean"),
    customers=("customer_id", "nunique")
)

channel_summary

In [ ]:
#7. Visualize :-

plt.figure(figsize=(8,5))
sns.scatterplot(
    data=channel_summary,
    x="customers",
    y="avg_clv",
    hue=channel_summary.index,
    s=120
)
plt.title("Acquisition Channel: Volume vs Value")
plt.show()

### ***-: CONCLUSION :-***

*Advanced Strategic Insights*

- Signup cohorts with stronger early retention consistently generate higher lifetime value.
- Customers who progress to repeat purchase stages contribute significantly more revenue.
- Acquisition channels differ in quality, not just scale, emphasizing the need for CLV-based channel evaluation.
- Early lifecycle interventions (onboarding, first purchase experience) have a compounding effect on long-term revenue.

### ***-: BUSINESS DELIVERABLES & DATA EXPORTS :-***

In [ ]:
# Deliverable 1: High-Value Customers List

high_value_customers = clv_df[
    clv_df["clv_segment"].isin(["High Value", "VIP"])
].sort_values("CLV", ascending=False)

high_value_customers.head()

In [ ]:
# Export to CSV :

high_value_customers.to_csv(
    "high_value_customers_clv.csv",
    index=False
)

In [ ]:
# Deliverable 2: Retention-Risk Customers

retention_risk_customers = clv_df[
    (clv_df["total_orders"] <= 1) &
    (clv_df["customer_lifetime_days"] <= 30)
]

retention_risk_customers.head()

In [ ]:
# Export to CSV :

retention_risk_customers.to_csv(
    "retention_risk_customers.csv",
    index=False
)

In [ ]:
# Deliverable 3: Channel Performance Summary

channel_performance = channel_summary.reset_index()
channel_performance

In [ ]:
# Export to CSV :

channel_performance.to_csv(
    "channel_clv_performance.csv",
    index=False
)

In [ ]:
# Deliverable 4: Funnel Drop-off Summary

funnel_export = funnel_table.reset_index().rename(
    columns={"index": "funnel_stage"}
)

funnel_export

In [ ]:
# Export to CSV :

funnel_export.to_csv(
    "funnel_conversion_summary.csv",
    index=False
)

In [ ]:
# Download CSVs to Drive :

from google.colab import files

files.download('high_value_customers_clv.csv')
files.download('retention_risk_customers.csv')
files.download('channel_clv_performance.csv')
files.download('funnel_conversion_summary.csv')

### ***-: FINAL CONCLUSION :-***

*This project demonstrates a comprehensive customer analytics workflow by integrating Customer Lifetime Value (CLV), cohort retention analysis, and funnel conversion analysis into a unified business perspective.*

*CLV analysis revealed a highly skewed revenue distribution, where a small segment of high-value and VIP customers contributes a disproportionate share of total revenue. Cohort analysis highlighted significant early-stage churn, emphasizing the importance of first-month engagement for long-term customer retention and value creation.*

*Funnel analysis identified major drop-offs during early stages of the customer journey, particularly between visit and signup, as well as limited repeat purchase behavior. When combined, these insights show that improving early lifecycle engagement and repeat purchase strategies can substantially increase overall customer lifetime value.*

*Overall, this project illustrates how businesses can move beyond surface-level metrics and leverage customer behavior, retention patterns, and value modeling to drive data-informed growth, retention, and marketing strategies.*



---

